In [51]:
import sqlite3
import json

def transform_raw_to_clean():
    # Connect to the existing database
    conn = sqlite3.connect("todos.db")
    cursor = conn.cursor()

    # Create the clean_todos table with the desired schema
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS clean_todos (
            id INTEGER PRIMARY KEY,
            user_id INTEGER,
            title TEXT,
            completed BOOLEAN
        )
    """)

    # Select all raw_data rows from raw_todos table
    cursor.execute("SELECT raw_data FROM raw_todos")
    rows = cursor.fetchall()

    # Iterate through each raw JSON text block
    for (raw_json_text,) in rows:
        try:
            # Parse the JSON string into Python objects (list of dicts)
            todos = json.loads(raw_json_text)

            # Insert each todo item into clean_todos table
            for todo in todos:
                cursor.execute("""
                    INSERT OR IGNORE INTO clean_todos (id, user_id, title, completed) 
                    VALUES (?, ?, ?, ?)
                """, (
                    todo.get('id'),
                    todo.get('userId'),
                    todo.get('title'),
                    todo.get('completed')
                ))

        except json.JSONDecodeError as e:
            print(f"JSON parsing error, skipping record: {e}")
            # Could add logging or move on; for now, just skip bad records

    # Commit all inserts
    conn.commit()
    conn.close()
    print("Transformation complete. Clean data loaded into 'clean_todos'.")

if __name__ == "__main__":
    transform_raw_to_clean()

Transformation complete. Clean data loaded into 'clean_todos'.
